In [2]:
# import all the E9 stuff
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1,
    "C:\\Users\\ken92\\Documents\\Studies\\E5\\simulation\\E9_simulations")
from E9_fn.E9_constants import *
from E9_fn import util

In [3]:
test_value = hbar / lambda_sw